### 데이터 집계
#### 수집한 데이터를 바탕으로 평균이나 합 등을 구하여 의미있는 값을 도출해내는 것
#### 전체 데이터를 요약, 정리하여 볼 수 있기 때문에 데이터 분석이 편해짐.

# groupby 메서드로 평균값 구하기

In [1]:
# 갭마인더 데이터
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep='\t')

In [2]:
# year 열을 기준으로 데이터 그룹화, lifeExp 열의 평균 구하기
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 분할-반영-결합 과정 살펴보기

In [3]:
# 분할
# - 연도별로 데이터를 나누는 과정 

years = df.year.unique()
print(years)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]


In [5]:
# 반영
# - 연도별로 평균값

# 1. 연도별로 데이터 추출
y1952 = df.loc[df.year==1952, :]
print(y1952.head())
# 2. 평균값 구하기
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)

        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053
49.05761971830987


In [ ]:
# 결합
# - 연도별로 구한 평균값을 합침
df2 = pd.DataFrame({"year":[1952, 1957, 1962, 2007], 
                   "":[y1952_mean, y1957_mean, y1962_mean, y2007_mean]})

### groupby 메서드와 함께 사용하는 집계 메서드
- count : 누락값을 제외한 데이터 수 반환
- size : 누락값을 포함한 데이터 수 반환
- mean : 평균값
- std : 표준편차
- min : 최솟값
- quantile(q=0.25) : 백분위수 25%
- quantile(q=0.50) : 백분위수 50%
- quantile(q=0.75) : 백분위수 75%
- max : 최댓값
- sum : 전체합
- var : 분산
- sem : 평균의 표준편차
- describe : 데이터 수, 평균, 표준편차, 최소값, 백분위수(25, 50, 75%), 최댓값
- first : 첫번째 행
- last : 마지막 행
- nth : n번째 행

# 평균값을 구하는 사용자 함수와 groupby 메서드

In [6]:
# 평균값을 구하는 사용자 함수
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum+=value
    return sum/n

In [7]:
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [8]:
# 두 개의 인자값을 받아 처리하는 사용자 함수
def my_mean_diff(values, diff_value):
    n = len(values)
    sum=0
    for value in values:
        sum +=value
    mean = sum/n
    return mean-diff_value

In [9]:
# 연도별 평균 수명 - 전체 평균 수명
global_mean = df.lifeExp.mean()
print(global_mean)

agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value = global_mean)
print(agg_mean_diff)

59.47443936619713
year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


# 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [11]:
# 여러개의 집계 메서드를 한번에 사용하기

# 리스트 사용
import numpy as np
gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

      count_nonzero       mean        std
year                                     
1952          142.0  49.057620  12.225956
1957          142.0  51.507401  12.231286
1962          142.0  53.609249  12.097245
1967          142.0  55.678290  11.718858
1972          142.0  57.647386  11.381953
1977          142.0  59.570157  11.227229
1982          142.0  61.533197  10.770618
1987          142.0  63.212613  10.556285
1992          142.0  64.160338  11.227380
1997          142.0  65.014676  11.559439
2002          142.0  65.694923  12.279823
2007          142.0  67.007423  12.073021


In [13]:
# 딕셔너리 사용
gdf_dict = df.groupby('year').agg({'lifeExp':'mean','pop':'median','gdpPercap':'median'})
print(gdf_dict)

        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334741
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
1992  64.160338   8688686.5  4386.085502
1997  65.014676   9735063.5  4781.825478
2002  65.694923  10372918.5  5319.804524
2007  67.007423  10517531.0  6124.371109


# 표준 점수 계산하기

In [15]:
# 표준 점수 : 데이터의 평균 - 표준편차
# 평균값 -> 0, 표준편차 -> 1이 됨

# 표준점수를 계산하는 함수
def my_zscore(x):
    return (x - x.mean())/x.std()

In [17]:
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


# 누락값을 평균값으로 처리하기

In [18]:
import seaborn as sns
import numpy as np

np.random.seed(42)

tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4],'total_bill'] = np.NaN
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [19]:
# 성별로 그룹화, 각 열의 데이터 수 구하기
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


In [20]:
# 성별을 구분하여 total_bill 평균 구하기
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [21]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


# 데이터 필터링 사용하기 ─ filter 메서드

In [22]:
tips = sns.load_dataset('tips')
print(tips.shape)

(244, 7)


In [24]:
# size 열의 데이터수 확인
print(tips['size'].value_counts())

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64


In [27]:
# 30번 이상의 주문이 있는 테이블만 추리고 싶음
tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count()>=30)
print(tips_filtered.shape)
print(tips_filtered['size'].value_counts())

(231, 7)
2    156
3     38
4     37
Name: size, dtype: int64


# 그룹 오브젝트 저장하여 살펴보기

In [28]:
# 그룹 오브젝트 : groupby() 메서드의 결과값

tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [29]:
grouped = tips_10.groupby('sex')
print(grouped)

In [30]:
print(grouped.groups)

{'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64'), 'Female': Int64Index([198, 124, 101], dtype='int64')}


# 그룹 오브젝트의 평균 구하기

In [31]:
# 파이썬은 자동으로 계산 할 수 있는 열을 골라주는 기능을 제공함
avg = grouped.mean()
print(avg)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000


In [32]:
print(tips_10.columns)

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')


# 그룹 오브젝트에서 데이터 추출하고 반복하기

In [33]:
# get_group() : 그룹 오브젝트에서 특정 데이터만 추출
female = grouped.get_group('Female')
print(female)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


In [34]:
# 반복문 사용하기
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


# 그룹 오브젝트 계산하고 살펴보기

In [35]:
# 여러 열을 사용해 그룹 오브젝트 만들고 꼐산하기
bill_sex_time = tips_10.groupby(['sex','time'])
group_avg = bill_sex_time.mean()

print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [36]:
# MultiIndex : 여러 인덱스를 담고 있는 인덱스
print(group_avg.index)

MultiIndex(levels=[['Male', 'Female'], ['Lunch', 'Dinner']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['sex', 'time'])


In [37]:
# reset_index() : 데이터프레임의 인덱스 새로 부여하기
group_method = tips_10.groupby(['sex','time']).mean().reset_index()
print(group_method)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000


In [38]:
# as_index = False : reset_index()
group_param = tips_10.groupby(['sex','time'], as_index=False).mean()
print(group_param)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
